In [ ]:
import pandas as pd
from selenium import webdriver  
import re  
import json  
import time 
from bs4 import BeautifulSoup
import re

options = webdriver.ChromeOptions()  
options.add_argument('disable-infobars')  
driver = webdriver.Chrome(chrome_options=options)  
comments=[]

In [100]:
### Taobao
page_url = 'https://item.taobao.com/item.htm?spm=a230r.1.14.69.63be6dc1IRZhGB&id=563566970802&ns=1&abbucket=11#detail'
search_url = 'https://s.taobao.com/search?q=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%B1%85&type=p&tmhkh5=&spm=a21wu.241046-us.a2227oh.d100&from=sea_1_searchbutton&catId=100'
# search_url = 'https://s.taobao.com/search?q=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%B1%85&type=p&tmhkh5=&spm=a21wu.241046-us.a2227oh.d100&from=sea_1_searchbutton&catId=100&bcoffset=3&ntoffset=3&p4ppushleft=1%2C48&s=44'
# search_url = 'https://s.taobao.com/search?q=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%B1%85&type=p&tmhkh5=&spm=a21wu.241046-us.a2227oh.d100&from=sea_1_searchbutton&catId=100&bcoffset=0&ntoffset=0&p4ppushleft=1%2C48&s=88'
# search_url = 'https://s.taobao.com/search?q=%E6%99%BA%E8%83%BD%E5%AE%B6%E5%B1%85&type=p&tmhkh5=&spm=a21wu.241046-us.a2227oh.d100&from=sea_1_searchbutton&catId=100&bcoffset=-6&ntoffset=-6&p4ppushleft=1%2C48&s=176'
driver.get(search_url)

In [107]:
# payment count for all the deals in this page
a = driver.find_elements_by_class_name('deal-cnt')

In [146]:
b = driver.find_elements_by_class_name('pic')[0]

In [63]:
print('Total Rate:', driver.find_elements_by_id('J_RateCounter')[0].text)
print('Total Sold:', driver.find_elements_by_id('J_SellCounter')[0].text)
print('Title:', driver.find_elements_by_class_name('tb-main-title')[0].text)

('Total Rate:', u'1500')
('Total Sold:', u'713')
('Title:', u'\u7535\u52a8\u7a97\u5e18\u9065\u63a7\u81ea\u52a8\u667a\u80fd\u7a97\u5e18\u5bb6\u7528wifi\u5f00\u5408\u5e18\u8f68\u9053\u6746\u963f\u91cc\u667a\u80fd\u5bb6\u5c45\u7535\u673a')


In [64]:
## Tmall
url = 'https://detail.tmall.com/item.htm?spm=a230r.1.14.27.63be6dc1vFTyxA&id=536785566352&ns=1&abbucket=11'
driver.get(url)

print('Monthly Sold Unit:', driver.find_element_by_xpath("//li[@class='tm-ind-item tm-ind-sellCount']").find_element_by_class_name('tm-count').text)
print('Total Rate:', driver.find_element_by_xpath("//li[@class='tm-ind-item tm-ind-reviewCount canClick tm-line3']").find_element_by_class_name('tm-count').text)
print('Title:', driver.find_element_by_class_name('tb-detail-hd').text)

In [148]:
## beautiful soup

In [290]:
def scrape_search_page_ali(total_pgs, search_content):
    total_list = []
    for i in range(0,total_pgs):
        time.sleep(2)
        search_url = 'https://s.taobao.com/search?q={2}&type=p&tmhkh5=&spm=a21wu.241046-us.a2227oh.d100\
        &from=sea_1_searchbutton&catId=100&bcoffset={0}&ntoffset={0}&p4ppushleft=1%2C48&s={1}'.format(str(-3*i + 6), str(i*44), search_content)
        response = get(search_url)
    #     html_soup = BeautifulSoup(response.text, 'html.parser')

        html=response.content
        fileds=re.findall(r'g_page_config = (.*?)g_srp_loadCss()',html,re.S)
        fileds=''.join(fileds[0]).replace(';','')
        fileds=re.sub('\s','',fileds)

        json_acceptable_string = fileds.replace("'", "\"")
        d = json.loads(json_acceptable_string)
        total_list.extend([[item['view_price'], item['view_sales'], item['raw_title']] for item in d['mods']['itemlist']['data']['auctions']])
    return pd.DataFrame(total_list, columns = ['Price','Payment_Count','Title'])

In [428]:
def getstr(s, i):  
    html = get('https://search.jd.com/Search?keyword={0}&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&wq={0}&page={1}&s={2}&click=0'.format(s,i, (i - 1)*30 + 1)).text  
    soup = BeautifulSoup(html, 'lxml')  
    lis = soup.find_all("li", class_='gl-item')  
    str1 = ''  
    for li in lis:  
        data_pid = li.get("data-sku")  
        str1=str1+str(data_pid)+','  
    return str1  

In [430]:
Str = '智能家居'

# url = 'https://search.jd.com/Search?keyword=%E5%9C%9F%E8%B1%86&enc=utf-8'  
# content = get(url).text  
# selector = etree.HTML(content)  
# links = selector.xpath(".//*[@id='J_goodsList']/ul/li/div/div[1]/a/@href")  

# bb = selector.xpath(".//*[@id='J_goodsList']/ul/li/div/div[1]/a")[0]
def scrape_search_page_ali(total_pgs, Str):
    urlbase = 'https://search.jd.com/s_new.php?keyword=%s&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&wq=%s&stock=1&page=%d&s=28&scrolling=y&log_id=%s'
    all_results = []
    for i in range(1,total_pgs + 1):  
        header = {  
            'Referer':'https://search.jd.com',  
        }  
        content = get(urlbase%(Str,Str,i,getstr(Str, i)),headers=header).text  

        total_list = []
        pat_sta = re.compile('<a target="_blank" title="(.*?)" href="(.*?)"')  
        items = re.findall(pat_sta,content)  
        for k in items:
            if 'https' not in k[1]:
                id_code = k[1].replace(u'//item.jd.com/','').split('.')[0]
                pat_price = re.compile('<strong class="J_%s" (.*?)(\d+\.\d+)'%id_code)  
                price_ = re.findall(pat_price,content)[0][-1]

                pat_comments = re.compile('<strong><a id="J_comment_%s"(.*?)>(.*?)<'%id_code)  
                comments_ = re.findall(pat_comments,content)[0][-1]
                total_list.append(list(k) + [price_, comments_])
        all_results.extend(total_list)
    return pd.DataFrame(all_results, columns = ['Title','Web','Price','Comment_Count']).drop_duplicates()

In [433]:
df = scrape_search_page_ali(3, Str)